In [31]:
import sys

sys.path.append("../modules")

import pushift_wrap as rapi
import os
import requests
import time
import random
import json
from datetime import datetime

data_path = os.path.join('D:/', 'data', 'reddit')

sub_end = "https://api.pushshift.io/reddit/search/submission/"
subcomment_end = "https://api.pushshift.io/reddit/submission/comment_ids/"
comment_end = "https://api.pushshift.io/reddit/search/comment/"

In [16]:
#load existing data
filename = 'reddit_wallstreetbets_01012021-15032021_not-finished.json'

with open(os.path.join(data_path, filename), 'r', encoding = 'utf-8') as f:
    submissions = json.load(f)

In [32]:
start_time_epoch = int(datetime(2021,2,6,0,0).timestamp())
end_time_epoch = int(datetime(2021,2,8,0,0).timestamp())

fields_get = ['author', 'created_utc', 'domain', 'full_link', 'gildings', 'id', 'is_original_content', 'is_reddit_media_domain', 'locked', 
              'media_only', 'num_comments', 'num_crossposts', 'over_18', 'permalink', 'pinned', 'retrieved_on', 'score', 'selftext', 'spoiler', 
              'stickied', 'subreddit', 'subreddit_id', 'subreddit_subscribers', 'subreddit_type', 'thumbnail', 'title', 'total_awards_received', 
              'upvote_ratio', 'url', 'comments', 'is_meta', 'link_flair_richtext', 'gilded']

comment_fields_get = ['author', 'banned_at_utc', 'body', 'collapsed', 'collapsed_reason', 'comment_type', 'created_utc', 'edited', 'id', 'is_submitter',
                      'link_id', 'parent_id', 'permalink', 'retrieved_on', 'score', 'stickied', 'subreddit', 'subreddit_id', 'total_awards_received',
                      'associated_award', 'author_flair_richtext']

In [33]:
from_time_epoch = start_time_epoch

submissions = []
while from_time_epoch <= end_time_epoch:
    
    to_time_epoch = from_time_epoch + 86400 # 24 hours
    new_submissions = rapi.get_submissions(subreddit = 'wallstreetbets', before = to_time_epoch, after = from_time_epoch, q = "gamestop", num_comments = 50, fields = fields_get)
    submissions = submissions + new_submissions
    
    from_time_epoch = to_time_epoch
    
    sleep_time = random.uniform(0.03, 0.10)
    time.sleep(sleep_time)
    

In [34]:
len(submissions)

3

In [35]:
sum([submission.get('num_comments') for submission in submissions])

424

In [36]:
submissions[0].keys()

dict_keys(['author', 'created_utc', 'domain', 'full_link', 'gildings', 'id', 'is_meta', 'is_original_content', 'is_reddit_media_domain', 'link_flair_richtext', 'locked', 'media_only', 'num_comments', 'num_crossposts', 'over_18', 'permalink', 'pinned', 'retrieved_on', 'score', 'selftext', 'spoiler', 'stickied', 'subreddit', 'subreddit_id', 'subreddit_subscribers', 'subreddit_type', 'thumbnail', 'title', 'total_awards_received', 'upvote_ratio', 'url'])

In [37]:
#submissions[0]

In [38]:
from json import JSONDecodeError

for c, submission in enumerate(submissions, start = 1):
    progress = "|{0}| {1:.2f} %".format(("="*int(c/len(submissions) * 50)).ljust(50), c/len(submissions) * 100)
    if 'comments' in submission:
        print(progress, end = "\r")
        continue
    
    try:
        subid = submission.get('id')
        commentids = requests.get(f"{subcomment_end}{subid}").json().get('data')
    
        comments = rapi.get_comments(commentids)
        submission['comments'] = comments
    
        time.sleep(random.uniform(0.03, 0.10))
    except JSONDecodeError:
        print(progress, end = "\r")
        continue
        
    print(progress, end = "\r")

In [39]:
sum([len(submission.get('comments')) for submission in submissions if 'comments' in submission])

404

In [40]:
filename = 'reddit_wallstreetbets-gamestop_06022021-08022021.json'

with open(os.path.join(data_path, 'wsb', filename), 'w', encoding = 'utf-8') as f:
    json.dump(submissions, f)